In [1]:
import numpy as np
import re
import pandas as pd
import geopandas as gpd
import gdown
import ee
import google
import os
import geemap

from tobler.util import h3fy
from ee_jupyter.ipyleaflet import Map

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BX3Wosqc0q9MpigKESYiyn53Y8qVpSLbl-JQj7SgHfQiJpjr3Ho81U

Successfully saved authorization token.


In [79]:
temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate('2010-01-01', '2019-12-31').select('temperature_2m').filter(ee.Filter.bounds(juneau.geometry()))


In [80]:
temp.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'temperature_2m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
 'version': 1680765300915096,
 'id': 'ECMWF/ERA5_LAND/HOURLY/20100101T00',
 'properties': {'system:time_start': 1262304000000,
  'hour': 0,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1262307600000,
  'system:asset_size': 395595778,
  'system:index': '20100101T00'}}

In [152]:
julianDayList = ee.List.sequence(1, 365)

def temporalReduction(day):
    day_temp = temp.filter(ee.Filter.dayOfYear(day, day))
    
    collection = []
    
    for i in range(24):
        hour = day_temp.filter(ee.Filter.eq('hour', i))
        average = hour.mean()
        collection.append(average)
        
    return ee.ImageCollection(collection)

temp_10year_average = ee.ImageCollection(julianDayList.map(temporalReduction)).flatten()

In [127]:
# julianDayList = ee.List.sequence(1, 1)

# def temporalReduction(day):
#     day_temp = temp.filter(ee.Filter.dayOfYear(day, day))
    
#     collection = []
    
#     for i in range(24):
#         hour = day_temp.filter(ee.Filter.eq('hour', i))
#         average = hour.mean()
#         collection.append(average)
        
#     return ee.ImageCollection(collection)

# temp_10year_average = ee.ImageCollection(julianDayList.map(temporalReduction)).flatten()

In [128]:
from ee_jupyter.ipyleaflet import Map

In [129]:
buildingsRaster.gt(0).projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [139]:
#map1 = Map(center=[64.835365, -147.776749], zoom=10)

buildings = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_AK')

image = ee.Image(temp_10year_average.first())

buildingsRaster = ee.Image().byte().paint(featureCollection = buildings, color = 1)
# // Get information about the ERA5-Land projection.
imageProjection = image.projection()

#     // Force the next reprojection to aggregate instead of resampling.
#     // The reduce resolution returns the fraction of the ERA-5 Land pixel
#     // that's covered by 30 meter forest pixels.  Convert to area
#     // after the reduceResolution() call.
#     // Request the data at the scale and projection of the ERA-5 Land image.

buildingsRaster = buildingsRaster.setDefaultProjection('EPSG:4326')

imageBuildings = buildingsRaster.gt(0).reduceResolution(reducer = ee.Reducer.mean(),maxPixels = 1024) \
    .multiply(ee.Image.pixelArea()).reproject(crs = imageProjection)

#map1.addLayer(imageBuildings)

CGAZ_ADM1 = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1')

# // Filter Comprehensive Global Administrative Zones ADM1 for AK
# // https://gee-community-catalog.org/projects/geoboundary/
ak = CGAZ_ADM1.filter(ee.Filter.eq('shapeISO', 'US-AK'))
juneau = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('COUNTYFP', '110')).filter(ee.Filter.eq('STATEFP', '02'))

# // Convert the feature collection to a geometry.
akGeometry = ak.geometry()
juneauGeometry = juneau.geometry()

# // Get the bounding box of the geometry.
akBounds = akGeometry.bounds()
juneauBounds = juneauGeometry.bounds()

# // Import ERA-5 Land
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filter(ee.Filter.date('2020-07-01', '2020-07-02'));

# // Get single image composite over AK, have to reproject because
# // mosiacs do not have default projection
image = dataset.max().clip(juneauGeometry).select('temperature_2m').reproject('EPSG:4326', None, 11132)

#print(imageProjection)

# // Import buildings
buildings = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_AK')

# // Rasterize buildings
buildingsRaster = ee.Image().byte().paint(featureCollection = buildings, color = 1)

# // Get coverage of buildings per ERA-5 Pixel
#     // Force the next reprojection to aggregate instead of resampling.
#     // The reduce resolution returns the fraction of the ERA-5 Land pixel
#     // that's covered by 100 meter building pixels.  Convert to area
#     // after the reduceResolution() call.
#     // Request the data at the scale and projection of the ERA-5 Land image.

# imageBuildings = buildingsRaster.setDefaultProjection('EPSG:4326', None, 100).reduceResolution(
#       reducer = ee.Reducer.mean(), maxPixels = 12393).multiply(ee.Image.pixelArea()).reproject('EPSG:4326', None, 11132);


#     // Force the next reprojection to aggregate instead of resampling.
#     // The reduce resolution returns the fraction of the ERA-5 Land pixel
#     // that's covered by 100 meter building pixels.  Convert to area
#     // after the reduceResolution() call.
#    // // Request the data at the scale and projection of the ERA-5 Land image.
imageBuildings = buildingsRaster.unmask(0).setDefaultProjection('EPSG:4326', None, 100).reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 12393).multiply(ee.Image.pixelArea()).reproject('EPSG:4326', None, 11132)


# // Export output
# Export.image.toAsset({
#   image: imageBuildings,
#   description: 'ERA-5LandBuildings',
#   assetId: 'ERA-5_LandBuildings',
#   scale: 11132,
#   region: akBounds,
#   maxPixels: 1e13
# });

In [140]:
type(imageBuildings)

ee.image.Image

In [141]:
centroids = imageBuildings.gt(0).sample(
  region = akGeometry,
  geometries =  True);

In [ ]:
# join centroids or imageBuildings mask
# small vs.          big
# update the above for juneau instead of alaska

In [ ]:
# updateMask() function mapped on 10year average 
# unmask 0
# reduce


In [149]:
blocks = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Blocks2020/Blocks2020.shp')
blocks = blocks[['NAME', 'STATE', 'BOROUGH', 'TRACT', 'BLOCK', 'FIPS', 'TOTALPOP', 'HOUSEUNITS', 'geometry']]
juneauBlocks = blocks[(blocks['BOROUGH'] == '110') & (blocks['TOTALPOP'] > 0)]
juneauBlocksFC = geemap.geopandas_to_ee(juneauBlocks)

In [150]:
juneauBlocksFC.size().getInfo()

380

In [151]:
areas = juneauBlocks['geometry'].area
areas.min(), areas.median(), areas.max()

(1055.8870653038284, 27503.41284678245, 2220401169.367476)

In [154]:
def maskTemp(image):
    image = ee.Image(image)
    return image.updateMask(imageBuildings).unmask(0)

maskedTemp = temp_10year_average.map(maskTemp)

In [156]:
type(maskedTemp)

ee.featurecollection.FeatureCollection

In [159]:
# spatial reduction according to juneauBlocksFC

def reduce_to_region(image):
    image = ee.Image(image)
    reduced_image = image.reduceRegions(collection=juneauBlocksFC, reducer=ee.Reducer.mean(), scale=1000)
    #reduced_image = reduced_image.set('date', image.get('julian_date'))
    return reduced_image

export = maskedTemp.map(reduce_to_region)

In [162]:
export1 = export.first()

In [166]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export2,
  'description': 'juneau export with lots of flattening',
  'folder': 'EarthEngine',
  'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [165]:
export2 = maskedTemp.map(reduce_to_region).flatten().flatten().flatten()

In [ ]:
var buildingsRaster = ee.Image().byte().paint({
    featureCollection: buildings,
    color: 1
});
// Get information about the ERA5-Land projection.
var imageProjection = image.projection();
var imageBuildings = buildingsRaster.gt(0)
    // Force the next reprojection to aggregate instead of resampling.
    .reduceResolution({
      reducer: ee.Reducer.mean(),
      maxPixels: 1024
    })
    // The reduce resolution returns the fraction of the ERA-5 Land pixel
    // that's covered by 30 meter forest pixels.  Convert to area
    // after the reduceResolution() call.
    .multiply(ee.Image.pixelArea())
    // Request the data at the scale and projection of the ERA-5 Land image.
    .reproject({
      crs: imageProjection
    });
Map.addLayer(imageBuildings)

Vectorize a ERA-5 image

Select first building per ERA-5 feature of feature collection (join-first)

Save this as a feature-collection to make it go faster

Calculate climate data at building

Get mean at block level per day – this is optional, I’d almost export it and aggregate after in python\

Export the feature collection

Can subset per borough, will run quickly


In [8]:
ak = ee.FeatureCollection("TIGER/2016/States").filter(ee.Filter.eq('GEOID', '02'))
ak.size().getInfo()

1

In [44]:
juneau = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('COUNTYFP', '110')).filter(ee.Filter.eq('STATEFP', '02'))
juneau.size().getInfo()

1

In [9]:
def clipToAK(image):
    return image.select('temperature_2m').clipToCollection(ak)

ak_temp = temp.map(clipToAK)

In [45]:
def clipToJ(image):
    return image.select('temperature_2m').clipToCollection(juneau)

juneau_temp = temp.map(clipToJ)

In [16]:
type(ak_temp)

ee.imagecollection.ImageCollection

In [24]:
primary = ak_temp
secondary = ak_buildings = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_AK')

joinFilter = ee.Filter.intersects(leftField = '.geo', rightField = '.geo', maxError = 1000)

joined = ee.Join.inner().apply(primary, secondary, joinFilter)

In [25]:
joined.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '20100101T00_0000000000000001e4f1',
 'properties': {'primary': {'type': 'Image',
   'bands': [{'id': 'temperature_2m',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [3601, 1801],
     'crs': 'EPSG:4326',
     'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
   'version': 1680765300915096,
   'id': 'ECMWF/ERA5_LAND/HOURLY/20100101T00',
   'properties': {'system:time_start': 1262304000000,
    'hour': 0,
    'system:time_end': 1262307600000,
    'system:asset_size': 395595778,
    'system:index': '20100101T00'}},
  'secondary': {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-149.72244675693338, 61.18462236002106],
      [-149.7223888271917, 61.18461162293288],
      [-149.72221500453884, 61.184577236342214],
      [-149.72217032396347, 61.18462876384541],
      [-149.7221524514845, 61.18465459499242],
      [-149.72238435560448, 61.184699756113424],
      [-149.7224467569

In [29]:
filtered_join = joined.distinct('primary')

In [32]:
type(filtered_join)

ee.featurecollection.FeatureCollection

In [46]:
joinFilter = ee.Filter.intersects(leftField = '.geo', rightField = '.geo', maxError = 1000)

joinedJ = ee.Join.inner().apply(juneau_temp, ak_buildings, joinFilter)

In [40]:
joinedJ.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '20100101T00_0000000000000001e4f1',
 'properties': {'primary': {'type': 'Image',
   'bands': [{'id': 'temperature_2m',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [3601, 1801],
     'crs': 'EPSG:4326',
     'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
   'version': 1680765300915096,
   'id': 'ECMWF/ERA5_LAND/HOURLY/20100101T00',
   'properties': {'system:time_start': 1262304000000,
    'hour': 0,
    'system:time_end': 1262307600000,
    'system:asset_size': 395595778,
    'system:index': '20100101T00'}},
  'secondary': {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-149.72244675693338, 61.18462236002106],
      [-149.7223888271917, 61.18461162293288],
      [-149.72221500453884, 61.184577236342214],
      [-149.72217032396347, 61.18462876384541],
      [-149.7221524514845, 61.18465459499242],
      [-149.72238435560448, 61.184699756113424],
      [-149.7224467569

In [47]:
joinedJ_distinct = joinedJ.distinct('primary')

In [48]:
joinedJ_distinct.size().getInfo()

EEException: An internal error has occurred (request: 2f2ec4ee-166a-4a7c-bc18-dc3c86bb10b1)